目标：
* 使用深度神经网络(DNNs)解决分类问题。
* 理解递归神经网络(RNNs)。

如果您有任何问题，请通过助教课时间、NTU COOL或发送邮件至mlta-2023-spring@googlegroups.com联系助教

# 一些实用函数
**固定随机数生成器种子以确保可重复性。**

In [1]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**用于从每个语音的原始MFCC特征预处理训练数据的辅助函数。**

一个音素可能跨越多个帧，并且依赖于过去和未来的帧。\
因此，我们连接相邻的音素进行训练以获得更高的准确性。**concat_feat**函数连接过去和未来的k帧（总共2k+1 = n帧），我们预测中心帧。

您可以随意修改数据预处理函数，但**不要丢弃任何帧**（如果您修改了函数，请记住检查帧数是否与课件中提到的相同）

In [2]:
import os
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n必须为奇数
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8):
    class_num = 41 # 注意：预计算值，不需要更改

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # 分割训练和验证数据
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


# 数据集

In [3]:
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# 模型
您可以随意修改模型的结构。

In [4]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

# 超参数

In [5]:
# 数据参数
concat_nframes = 3              # 要连接的帧数，n必须为奇数（总共2k+1 = n帧）
train_ratio = 0.75               # 用于训练的数据比例，其余用于验证

# 训练参数
seed = 1213                        # 随机种子
batch_size = 512                # 批次大小
num_epoch = 10                   # 训练轮数
learning_rate = 1e-4         # 学习率
model_path = './model.ckpt'     # 保存检查点的路径

# 模型参数
input_dim = 39 * concat_nframes # 模型的输入维度，您不应该更改此值
hidden_layers = 2               # 隐藏层数
hidden_dim = 64                # 隐藏层维度

# 数据加载器

In [6]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# 预处理数据
train_X, train_y = preprocess_data(split='train', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# 获取数据集
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# 删除原始特征以节省内存
del train_X, train_y, val_X, val_y
gc.collect()

# 获取数据加载器
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 2571


2571it [00:14, 181.62it/s]


[INFO] train set
torch.Size([1588590, 117])
torch.Size([1588590])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:02, 421.83it/s]


[INFO] val set
torch.Size([525078, 117])
torch.Size([525078])


# 训练

In [7]:
# 创建模型，定义损失函数和优化器
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # 训练
    model.train() # 将模型设置为训练模式
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # 获取概率最高的类别索引
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    
    # 验证
    model.eval() # 将模型设置为评估模式
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
            
            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # 获取概率最高的类别索引
            val_loss += loss.item()

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # 如果模型改进，在此轮保存检查点
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')


100%|██████████| 1026/1026 [00:03<00:00, 264.67it/s]


[001/010] Train Acc: 0.39279 Loss: 2.21685 | Val Acc: 0.43877 loss: 1.97101
saving model with acc 0.43877


100%|██████████| 1026/1026 [00:03<00:00, 267.00it/s]


[002/010] Train Acc: 0.45006 Loss: 1.92895 | Val Acc: 0.45998 loss: 1.87939
saving model with acc 0.45998


100%|██████████| 1026/1026 [00:04<00:00, 247.78it/s]


...

[008/010] Train Acc: 0.49323 Loss: 1.73994 | Val Acc: 0.49616 loss: 1.72260
saving model with acc 0.49616


100%|██████████| 1026/1026 [00:03<00:00, 271.37it/s]


[009/010] Train Acc: 0.49606 Loss: 1.72781 | Val Acc: 0.49852 loss: 1.71150
saving model with acc 0.49852


100%|██████████| 1026/1026 [00:03<00:00, 286.58it/s]

[010/010] Train Acc: 0.49840 Loss: 1.71736 | Val Acc: 0.50044 loss: 1.70057
saving model with acc 0.50044


In [8]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

23

# 测试
创建测试数据集，并从保存的检查点加载模型。

In [9]:
# 加载数据
test_X = preprocess_data(split='test', feat_dir='/kaggle/input/ml2023spring-hw2/libriphone/feat', phone_path='/kaggle/input/ml2023spring-hw2/libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:04, 182.29it/s]

[INFO] test set
torch.Size([527364, 117])


In [10]:
# 加载模型
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

进行预测。

In [11]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # 获取概率最高的类别索引
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 1031/1031 [00:02<00:00, 502.27it/s]


将预测结果写入CSV文件。

运行完此代码块后，从左侧文件栏下载`prediction.csv`文件并提交到Kaggle。

In [12]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))